<a href="https://colab.research.google.com/github/ThatiFlowers/Treinamento/blob/main/CRUD_Py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3

def conexao_bd():
    conexao = sqlite3.connect("cafe_porquinho.db")
    return conexao

def create_tables():
    conexao = conexao_bd()
    cursor = conexao.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS produtos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT NOT NULL,
        preco REAL NOT NULL,
        categoria TEXT NOT NULL
    )
    """)

    conexao.commit()
    conexao.close()

create_tables()


In [ ]:
from os import name
# Criando a classe de produtos/ repositório(domínio)
class Produto:
    def __init__(self,nome,preco,categoria):
        self.nome = nome
        self.preco = preco
        self.categoria = categoria
    def __repr__(self):
        return f"Produto(id={self.id}, nome='{self.nome}', preco={self.preco}, categoria='{self.categoria}')"

In [ ]:
# Crie uma pasta 'repository'no ambiente notebook
!mkdir -p repository

In [ ]:

import sqlite3
import logging
from typing import List, Optional
from models.product_model import ProductModel
from db_utils import conexao_bd

class ProductRepository:
    def __init__(self):
        # Initialize the connection, but establish it per operation to ensure fresh state
        pass

    def create_tables(self):
        conexao = conexao_bd()
        cursor = conexao.cursor()
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            price REAL NOT NULL,
            stock INTEGER NOT NULL
        )
        """)
        conexao.commit()
        conexao.close()

    def insert(self, product: ProductModel) -> int:
        conexao = conexao_bd()
        cursor = conexao.cursor()
        try:
            cursor.execute("""
                INSERT INTO products (name, price, stock)
                VALUES (?, ?, ?)
            """, (product.name, product.price, product.stock))
            conexao.commit()
            product_id = cursor.lastrowid
            logging.debug(f"[PRODUCT-STORAGE] Product inserted with ID {product_id}")
            return product_id
        except Exception as error:
            logging.error(f"[PRODUCT-STORAGE] Fail to insert product: {error}")
            raise error
        finally:
            conexao.close()

    def get(self) -> List[ProductModel]:
        conexao = conexao_bd()
        cursor = conexao.cursor()
        try:
            cursor.execute("SELECT id, name, price, stock FROM products")
            rows = cursor.fetchall()
            ret = []
            for row in rows:
                product = ProductModel(
                    id=row[0],
                    name=row[1],
                    price=row[2],
                    stock=row[3]
                )
                ret.append(product)
            logging.debug(f"[PRODUCT-STORAGE] Retrieved {len(ret)} products")
            return ret
        except Exception as error:
            logging.error(f"[PRODUCT-STORAGE] Fail to get products: {error}")
            raise error
        finally:
            conexao.close()

    def clear_products(self):
        conexao = conexao_bd()
        cursor = conexao.cursor()
        try:
            cursor.execute("DELETE FROM products")
            conexao.commit()
            logging.debug("[PRODUCT-STORAGE] All products cleared")
        except Exception as error:
            logging.error(f"[PRODUCT-STORAGE] Fail to clear products: {error}")
            raise error
        finally:
            conexao.close()

In [ ]:
def cadastrar_produto(produto):
    conexao = conexao_bd()
    cursor = conexao.cursor()

    cursor.execute(
        "INSERT INTO produtos (nome, preco, categoria) VALUES (?, ?, ?)",
        (produto.nome, produto.preco, produto.categoria)
    )

    conexao.commit()
    conexao.close()


In [ ]:
def limpar_produtos():
    conexao = conexao_bd()
    cursor = conexao.cursor()
    cursor.execute("DELETE FROM produtos")
    conexao.commit()
    conexao.close()



In [ ]:
#teste

produto1 = Produto("cocavinho",12, "bebida")
produto2 = Produto("bolorosa",12.59,"comida")
produto3 = Produto("brownie", 11, "comida")
cadastrar_produto(produto1)
cadastrar_produto(produto2)
cadastrar_produto(produto3)

In [ ]:
def listar_produtos():
  conexao = conexao_bd()
  cursor = conexao.cursor()
  cursor.execute("select * from produtos")
  rows = cursor.fetchall()
  conexao.close()
  return rows

print("Produtos cadastrados:")
for produto_data in listar_produtos():
    # Assuming the order is id, nome, preco, categoria based on your create_tables()
    print(f"ID: {produto_data[0]}, Nome: {produto_data[1]}, Preço: {produto_data[2]}, Categoria: {produto_data[3]}")

Produtos cadastrados:
ID: 1, Nome: cocavinho, Preço: 12.0, Categoria: bebida
ID: 2, Nome: bolorosa, Preço: 12.59, Categoria: comida
ID: 3, Nome: brownie, Preço: 11.0, Categoria: comida


In [ ]:
for p in listar_produtos():
    print(p[1], p[2], p[3])

cocavinho 12.0 bebida
bolorosa 12.59 comida
brownie 11.0 comida


In [ ]:
# covertendo em uma lista de produtos
def converter_rows_para_produtos(rows):
    lista_de_produtos = []
    for row in rows:
        # formato: (id, nome, preco, categoria)
        p = produto(row[1], row[2], row[3])
        lista_de_produtos.append(p)
    return lista_de_produtos

def produtos_id(id):
    conexao = conexao_bd()
    cursor = conexao.cursor()

    cursor.execute("select * from produtos where id =?", (id,))
    row = cursor.fetchall()
    conexao.close()
    if row:
        p = produto(row[1], row[2], row[3])
        return p
    return None # Retorna "none" se o produto for encontrado com ID conhecido

In [ ]:
from typing import Optional
import logging

class ProductModel:
    def __init__(self, name: str, price: float, stock: int, id: Optional[int] = None):
        self.id = id
        self.name = name
        self.price = price
        self.stock = stock

    def __repr__(self):
        return f"ProductModel(id={self.id}, name='{self.name}', price={self.price}, stock={self.stock})"

def get(self) -> list:
    try:
        cursor = self.conn.cursor()
        cursor.execute("SELECT id, name, price, stock FROM products")

        rows = cursor.fetchall()
        ret = []

        for row in rows:
            product = ProductModel(
                id=row[0],
                name=row[1],
                price=row[2],
                stock=row[3]
            )
            ret.append(product)

        logging.debug(f"[PRODUCT-STORAGE] Retrieved {len(ret)} products")
        return ret
    except Exception as error:
        logging.error(f"[PRODUCT-STORAGE] Fail to get products: {error}")
        raise error

In [ ]:
import logging
from models.product_model import ProductModel

def insert(self, product: ProductModel) -> int:
    try:
        cursor = self.conn.cursor()
        cursor.execute("""
            INSERT INTO products (name, price, stock)
            VALUES (?, ?, ?)
        """, (product.name, product.price, product.stock))

        self.conn.commit()
        product_id = cursor.lastrowid

        logging.debug(f"[PRODUCT-STORAGE] Product inserted with ID {product_id}")
        return product_id
    except Exception as error:
        logging.error(f"[PRODUCT-STORAGE] Fail to insert product: {error}")
        raise error

In [ ]:

import sqlite3

def conexao_bd():
    conexao = sqlite3.connect("cafe_porquinho.db")
    return conexao


Writing db_utils.py


In [ ]:

import sqlite3

def conexao_bd():
    conexao = sqlite3.connect("cafe_porquinho.db")
    return conexao


Overwriting db_utils.py


In [ ]:
# Crie uma pasta 'models'no ambiente notebook
!mkdir -p models

In [ ]:
%%writefile models/product_model.py
from typing import Optional

class ProductModel:
    def __init__(self, name: str, price: float, stock: int, id: Optional[int] = None):
        self.id = id
        self.name = name
        self.price = price
        self.stock = stock

    def __repr__(self):
        return f"ProductModel(id={self.id}, name='{self.name}', price={self.price}, stock={self.stock})"

Overwriting models/product_model.py


In [ ]:
pip install Flask

In [ ]:

from flask import flask
import sqlite3

# Database connection function (from WEFJx95LjIDo)
def conexao_bd():
    conexao = sqlite3.connect("cafe_porquinho.db")
    return conexao

# Produto class definition (from TaQ1X8qxkCjR)
class Produto:
    def __init__(self,nome,preco,categoria):
        self.nome = nome
        self.preco = preco
        self.categoria = categoria
    def __repr__(self):
        # Note: The original __repr__ assumes an 'id' attribute, which is not set during __init__.
        # This might cause an AttributeError if an ID is not explicitly assigned to the object.
        return f"Produto(id={getattr(self, 'id', 'N/A')}, nome='{self.nome}', preco={self.preco}, categoria='{self.categoria}')"

# Function to register a product (from 5BT5e7cBm0Iv)
def cadastrar_produto(produto):
    conexao = conexao_bd()
    cursor = conexao.cursor()
    cursor.execute(
        "INSERT INTO produtos (nome, preco, categoria) VALUES (?, ?, ?)",
        (produto.nome, produto.preco, produto.categoria)
    )
    conexao.commit()
    conexao.close()

# Function to list products (from 15ee9bbd)
def listar_produtos():
    conexao = conexao_bd()
    cursor = conexao.cursor()
    cursor.execute("select * from produtos")
    rows = cursor.fetchall()
    conexao.close()
    return rows

# Initialize Flask application
app = Flask(__name__)

# Further Flask routes will be added here

In [ ]:

from flask import Flask, request, jsonify
import sqlite3

# Database connection function
def conexao_bd():
    conexao = sqlite3.connect("cafe_porquinho.db")
    return conexao

# Produto class definition
class Produto:
    def __init__(self,nome,preco,categoria):
        self.nome = nome
        self.preco = preco
        self.categoria = categoria
    def __repr__(self):
        return f"Produto(id={getattr(self, 'id', 'N/A')}, nome='{self.nome}', preco={self.preco}, categoria='{self.categoria}')"

# Function to register a product
def cadastrar_produto(produto):
    conexao = conexao_bd()
    cursor = conexao.cursor()
    cursor.execute(
        "INSERT INTO produtos (nome, preco, categoria) VALUES (?, ?, ?)",
        (produto.nome, produto.preco, produto.categoria)
    )
    conexao.commit()
    conexao.close()

# Function to list products
def listar_produtos():
    conexao = conexao_bd()
    cursor = conexao.cursor()
    cursor.execute("select * from produtos")
    rows = cursor.fetchall()
    conexao.close()
    return rows

# Initialize Flask application
app = Flask(__name__)

# Define GET endpoint for /produtos
@app.route('/produtos', methods=['GET'])
def get_produtos():
    produtos_db = listar_produtos()
    produtos_json = []
    for p_data in produtos_db:
        produtos_json.append({
            'id': p_data[0],
            'nome': p_data[1],
            'preco': p_data[2],
            'categoria': p_data[3]
        })
    return jsonify(produtos_json)


Overwriting app.py


In [ ]:

from flask import Flask, request, jsonify
import sqlite3

# Database connection function
def conexao_bd():
    conexao = sqlite3.connect("cafe_porquinho.db")
    return conexao

# Produto class definition
class Produto:
    def __init__(self,nome,preco,categoria):
        self.nome = nome
        self.preco = preco
        self.categoria = categoria
    def __repr__(self):
        return f"Produto(id={getattr(self, 'id', 'N/A')}, nome='{self.nome}', preco={self.preco}, categoria='{self.categoria}')"

# Function to register a product
def cadastrar_produto(produto):
    conexao = conexao_bd()
    cursor = conexao.cursor()
    cursor.execute(
        "INSERT INTO produtos (nome, preco, categoria) VALUES (?, ?, ?)",
        (produto.nome, produto.preco, produto.categoria)
    )
    conexao.commit()
    conexao.close()

# Function to list products
def listar_produtos():
    conexao = conexao_bd()
    cursor = conexao.cursor()
    cursor.execute("select * from produtos")
    rows = cursor.fetchall()
    conexao.close()
    return rows

# Initialize Flask application
app = Flask(__name__)

# Define GET and POST endpoint for /produtos
@app.route('/produtos', methods=['GET', 'POST'])
def handle_produtos():
    if request.method == 'GET':
        produtos_db = listar_produtos()
        produtos_json = []
        for p_data in produtos_db:
            produtos_json.append({
                'id': p_data[0],
                'nome': p_data[1],
                'preco': p_data[2],
                'categoria': p_data[3]
            })
        return jsonify(produtos_json)

    elif request.method == 'POST':
        data = request.get_json()
        if not data or not all(k in data for k in ('nome', 'preco', 'categoria')):
            return jsonify({'error': 'Dados incompletos para o produto'}), 400

        nome = data['nome']
        preco = data['preco']
        categoria = data['categoria']

        try:
            novo_produto = Produto(nome, preco, categoria)
            cadastrar_produto(novo_produto)
            return jsonify({'message': 'Produto cadastrado com sucesso', 'produto': {'nome': nome, 'preco': preco, 'categoria': categoria}}), 201
        except Exception as e:
            return jsonify({'error': str(e)}), 500


Overwriting app.py


In [ ]:
from app import app

if __name__ == '__main__':
    # Run the Flask app on host='0.0.0.0' to make it accessible externally
    # within the Colab environment. The default port is 5000.
    # For external access outside Colab, consider using ngrok or similar tools.
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app 'app'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
